## Load EPA FRS datasets to get facility data and NAICS/SIC mappings

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import sys

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

# import sys
# sys.path.append('../src/')

In [ ]:
# import re
import pandas as pd
import numpy as np

In [ ]:
from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

Create an S3 resource for the bucket holding source data

In [ ]:
import boto3
s3_resource = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_resource.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create an S3 client.  We will user later when we write out data and metadata
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

schemaname = 'epa_frs'
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(df)
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, '/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp))
    # df.to_parquet('/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, index=False))
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp),
        Filename='/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    print('dropping table: ' + tablename)
    cur.fetchall()
    
    schema = create_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/{uuid}/{timestamp}'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename,uuid=ingest_uuid,timestamp=timestamp)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

Load EPA FRS data file using pandas *read_csv* and using *ingest_uuid* as the global UUID for this ingestion

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

The schemaname for this table is `epa_frs`.  Dunno if it's a good idea or simply redundant to prefix the tablename with `epa_frs_` or `epa_` or `frs_` or...

In [ ]:
custom_meta_content = {
    'title': 'EPA FRS Dataset',
    'author': 'US EPA',
    'contact': 'frs_support@epa.gov',
    'description': 'desc',
    'release_date': '20211104',
    # How should we describe our transformative step here?
}

naics_metadata_df = None
custom_meta_fields = None

Prepare GLEIF matching data

In [ ]:
gleif_file = s3_resource.Object(os.environ['S3_LANDING_BUCKET'],f'mtiemann-GLEIF/EPA_FRS-matches.csv')
gleif_file.download_file(f'/tmp/EPA_FRS-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/EPA_FRS-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = { k:v for k, v in zip(gleif_df['ORG_NAME'], gleif_df.LEI) }

In [ ]:
custom_date_parser = lambda x: x # pd.to_datetime (x, format='%d-%m-%y')

usecols_dict = {
    'ORGANIZATION': ["REGISTRY_ID","PGM_SYS_ACRNM","INTEREST_TYPE","AFFILIATION_TYPE","START_DATE","END_DATE","ORG_NAME","ORG_TYPE","DUNS_NUMBER","DIVISION_NAME","EIN","MAILING_ADDRESS","SUPPLEMENTAL_ADDRESS","CITY_NAME","STATE_CODE","POSTAL_CODE","COUNTRY_NAME"],
    'FACILITY': ["REGISTRY_ID","PRIMARY_NAME","LOCATION_ADDRESS","CITY_NAME","COUNTY_NAME","STATE_CODE","COUNTRY_NAME","POSTAL_CODE","HUC_CODE","PGM_SYS_ACRNMS","LATITUDE83","LONGITUDE83"],
    'NAICS': ["REGISTRY_ID","PGM_SYS_ACRNM","INTEREST_TYPE","NAICS_CODE","PRIMARY_INDICATOR","CODE_DESCRIPTION"],
    'SIC': ["REGISTRY_ID","PGM_SYS_ACRNM","INTEREST_TYPE","SIC_CODE","PRIMARY_INDICATOR","CODE_DESCRIPTION"],
}

timestamp = ''
for name in usecols_dict.keys():
    bObj = bucket.Object(f'EPA/national_combined-20211104/NATIONAL_{name}_FILE.CSV')
    timestamp = max(bObj.last_modified.isoformat(), timestamp)
    
for name in usecols_dict.keys():
    bObj = bucket.Object(f'EPA/national_combined-20211104/NATIONAL_{name}_FILE.CSV')
    bObj.download_file(f'/tmp/foo{timestamp}.csv')

    # It takes almost a minute (!) to load nearly 600K rows of data
    dtype = { col:"string" for col in usecols_dict[name] }
    if name == 'ORGANIZATION':
        parse_dates=['START_DATE', 'END_DATE']
    else:
        parse_dates = None
        if name == 'FACILITY':
            dtype["LATITUDE83"] = dtype["LONGITUDE83"] = 'float32'
    
    df = pd.read_csv(f'/tmp/foo{timestamp}.csv', sep=',', header=0, usecols=usecols_dict[name], dtype=dtype, parse_dates=parse_dates, date_parser=custom_date_parser, engine='c')
    os.unlink(f'/tmp/foo{timestamp}.csv')
    if name == 'ORGANIZATION':
        df['LEI'] = df['ORG_NAME'].map(gleif_dict)
        del(gleif_dict)
        df.EIN.apply(lambda x: x if pd.isna(x) else x.replace('-', ''))
        df.loc[(df.EIN>'999999999')|(df.EIN.str.len()!=9), 'EIN'] = '-1'
        df.EIN = df.EIN.astype('Int64')
        cols = df.columns.tolist()
        cols = [ cols[0], cols[-1]] + cols[1:-1]
        df = df[cols]
    df.info(verbose=True)
    tablename = name.lower()
    create_trino_pipeline (s3, schemaname, tablename, timestamp, df, custom_meta_fields, custom_meta_content)
    del(df)

Verify that we can restore data and metadata

In [ ]:
# Read the Parquet file into an Arrow table
obj = s3.get_object(
    Bucket=os.environ['S3_DEV_BUCKET'], 
    Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
)
restored_table = pq.read_table(io.BytesIO(obj['Body'].read()))
# Call the table’s to_pandas conversion method to restore the dataframe
# This operation uses the Pandas metadata to reconstruct the dataFrame under the hood
restored_df = restored_table.to_pandas()
# The custom metadata is accessible via the Arrow table’s metadata object
# Use the custom metadata key used earlier (taking care to once again encode the key as bytes)
restored_meta_json = restored_table.schema.metadata[custom_meta_key.encode()]
# Deserialize the json string to get back metadata
restored_meta = json.loads(restored_meta_json)
# Use the custom metadata fields key used earlier (taking care to once again encode the key as bytes)
restored_meta_json_fields = restored_table.schema.metadata[custom_meta_key_fields.encode()]
# Deserialize the json string to get back metadata
restored_meta_fields = json.loads(restored_meta_json_fields)

In [ ]:
restored_df

In [ ]:
restored_meta

In [ ]:
restored_meta_fields

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'epa_frs',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':ingest_uuid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta

# ??? Now what?

In [ ]:
cur.execute(f'select name from osc_datacommons_dev.sec_dera.sub where osc_datacommons_dev.sec_dera.sub.ein in (select distinct(osc_datacommons_dev.{schemaname}.organization.ein) from osc_datacommons_dev.{schemaname}.organization where EIN > 0)')
cur.fetchall()

In [ ]:
cur.execute(f'select name from osc_datacommons_dev.sec_dera.sub where osc_datacommons_dev.sec_dera.sub.lei in (select distinct(osc_datacommons_dev.{schemaname}.organization.lei) from osc_datacommons_dev.{schemaname}.organization where lei is not null)')
cur.fetchall()

In [ ]:
cur.execute(f'select name from osc_datacommons_dev.sec_dera.sub where osc_datacommons_dev.sec_dera.sub.name in (select distinct(osc_datacommons_dev.{schemaname}.organization.org_name) from osc_datacommons_dev.{schemaname}.organization)')
cur.fetchall()

In [ ]:
cur.execute(f'select distinct(osc_datacommons_dev.{schemaname}.organization.org_name) from osc_datacommons_dev.{schemaname}.organization where LEI is not null')
cur.fetchall()

In [ ]:
cur.execute(f'select distinct(osc_datacommons_dev.sec_dera.sub.name) from osc_datacommons_dev.sec_dera.sub where LEI is not null')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from osc_datacommons_dev.epa_frs.organization where LEI is not null')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from osc_datacommons_dev.sec_dera.sub where LEI is not null')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from osc_datacommons_dev.epa_ghgrp.parent_company where LEI is not null')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from (select distinct (lei) from osc_datacommons_dev.epa_ghgrp.parent_company where LEI is not null)')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from osc_datacommons_dev.epa_frs.organization where LEI is not null')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from (select distinct (lei) from osc_datacommons_dev.epa_frs.organization where LEI is not null)')
cur.fetchall()

In [ ]:
cur.execute ('select count (*) from osc_datacommons_dev.us_census.all_sector_survey_2017')
cur.fetchall()

In [ ]:
cur.execute ('show schemas in osc_datacommons_dev')
cur.fetchall()